In [ ]:
import sympy as sp
x,y,z = sp.symbols('x y z')
def hessian(f):
    return sp.Matrix([[f.diff(x).diff(x),f.diff(x).diff(y),f.diff(x).diff(z)],
                      [f.diff(y).diff(x),f.diff(y).diff(y),f.diff(y).diff(z)],
                      [f.diff(z).diff(x),f.diff(z).diff(y),f.diff(z).diff(z)]])
def devgrad(f):
    Gf = sp.Matrix([[f[0].diff(x),f[0].diff(y),f[0].diff(z)],
                    [f[1].diff(x),f[1].diff(y),f[1].diff(z)],
                    [f[2].diff(x),f[2].diff(y),f[2].diff(z)]])
    return Gf - Gf.trace()/3*sp.Matrix.eye(3)
def curl(f):
    Cf = sp.Matrix([[f[0,2].diff(y)-f[0,1].diff(z),f[0,0].diff(z)-f[0,2].diff(x),f[0,1].diff(x)-f[0,0].diff(y)],
                    [f[1,2].diff(y)-f[1,1].diff(z),f[1,0].diff(z)-f[1,2].diff(x),f[1,1].diff(x)-f[1,0].diff(y)],
                    [f[2,2].diff(y)-f[2,1].diff(z),f[2,0].diff(z)-f[2,2].diff(x),f[2,1].diff(x)-f[2,0].diff(y)]])
    return Cf
def symcurl(f):
    Cf = curl(f)
    return (Cf+Cf.transpose())/2
def div(f):
    return sp.Matrix([f[0,0].diff(x)+f[0,1].diff(y)+f[0,2].diff(z), 
                      f[1,0].diff(x)+f[1,1].diff(y)+f[1,2].diff(z),
                      f[2,0].diff(x)+f[2,1].diff(y)+f[2,2].diff(z)])
def divdiv(f):
    return (f[0,0].diff(x).diff(x) +
            f[1,0].diff(y).diff(x) +
            f[2,0].diff(z).diff(x) +
            f[0,1].diff(x).diff(y) +
            f[1,1].diff(y).diff(y) +
            f[2,1].diff(z).diff(y) +
            f[0,2].diff(x).diff(z) +
            f[1,2].diff(y).diff(z) +
            f[2,2].diff(z).diff(z) )
def simplifyMat(A):
    for i in range(A.rows):
        for j in range(A.cols):
            A[i,j] = A[i,j].simplify()
    return A
def ev(f,ax,ay,az):
    return f.subs(x,ax).subs(y,ay).subs(z,az)
def S(A):
    SA = A.transpose() - A.trace()*sp.Matrix.eye(3)
    return simplifyMat(SA)
def dt(A):
    At = sp.Matrix.eye(3)
    for i in range(A.rows):
        for j in range(A.cols):
            At[i,j] = A[i,j].diff(t)
    return At
def L2(A,B):
    return sp.integrate(sp.integrate(sp.integrate(
        (sp.hadamard_product(A,B)*sp.Matrix.ones(A.cols,A.rows)).trace(),
        (x,0,1)),(y,0,1)),(z,0,1))

In [ ]:
import re
def cppcode(f):
    s = sp.ccode(f.simplify())
    s = re.sub("sin","autodiff::detail::sin",s)
    s = re.sub("cos","autodiff::detail::cos",s)
    s = re.sub("exp","autodiff::detail::exp",s)
    s = re.sub("pow","autodiff::detail::pow",s)
    return s
ADB = "      return autodiff1st(\n        [](autodiff::real x, autodiff::real y, autodiff::real z)->autodiff::real {\n          return "
ADE = ";\n        },X,dx,dy,dz);\n"
def ADPrint(A):
    if (A.cols == 1):
        print("    if (i == 0) {\n",ADB,cppcode(A[0]),ADE,
              "    } else if (i == 1) {\n",ADB,cppcode(A[1]),ADE,
              "    } else {\n",ADB,cppcode(A[2]),ADE,
              "    };",sep='')
    else:
        print("    if (i == 0 && j == 0) {\n",ADB,cppcode(A[0,0]),ADE,
              "    } else if (i == 0 && j == 1) {\n",ADB,cppcode(A[0,1]),ADE,
              "    } else if (i == 0 && j == 2) {\n",ADB,cppcode(A[0,2]),ADE,
              "    } else if (i == 1 && j == 0) {\n",ADB,cppcode(A[1,0]),ADE,
              "    } else if (i == 1 && j == 1) {\n",ADB,cppcode(A[1,1]),ADE,
              "    } else if (i == 1 && j == 2) {\n",ADB,cppcode(A[1,2]),ADE,
              "    } else if (i == 2 && j == 0) {\n",ADB,cppcode(A[2,0]),ADE,
              "    } else if (i == 2 && j == 1) {\n",ADB,cppcode(A[2,1]),ADE,
              "    } else {\n",ADB,cppcode(A[2,2]),ADE,
              "    };",sep='')


In [ ]:
def print0(f):
    df = devgrad(f)
    print("  static double f(unsigned i, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(f)
    print("  }\n  static double df(unsigned i, unsigned j, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(df)
    print("  }")
def print1(f):
    df = symcurl(f)
    print("  static double f(unsigned i, unsigned j, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(f)
    print("  }\n  static double df(unsigned i, unsigned j, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(df)
    print("  }")
def print2(f):
    df = divdiv(f)
    print("  static double f(unsigned i, unsigned j, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(f)
    print("  }\n  static double df(const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    print(ADB,cppcode(df),ADE,sep='')
    print("  }")

In [ ]:
P0 = sp.Matrix([[(3*x**3+2*x**2-x+1)*(y**2+3*y-1)*(2*z**2-z+2)],
                [(5*x**2-x+1)*(4*y**3-y**2+2*y-1)*(3*z**2-z+3)],
                [(1*x**2-x+4)*(-y**2+3*y-1)*(-z**3-2*z**2-z+4)]])
P1 = sp.Matrix([[(2*x**2-x+1)*(y**2+3*y-1)*(2*z**2-z+2),(x**3-2*x**2-x+1)*(3*y-1)*(2*z**2-z+2),(2*x**3+x**2-x+1)*(y**2+3*y-1)*(z+2)],
                  [(x+1)*(4*y**3-y**2+2*y-1)*(3*z**2-z+3),(3*x**2-x+1)*(-y**2+1*y-1)*(3*z**2-z+3),(5*x**2-x+1)*(4*y**3-y**2+2*y-1)*(z-1)],
                  [(-x+4)*(-y**2+3*y-1)*(-z**3-2*z**2-z+3),(2*x**2-x+4)*(y-1)*(-z**3-2*z**2-z+4),-(2*x**2-x+1)*(y**2+3*y-1)*(2*z**2-z+2)-(3*x**2-x+1)*(-y**2+1*y-1)*(3*z**2-z+3)]])
P2 = sp.Matrix([[(3*x**3+2*x**2-x+1)*(3*y-1)*(z+2),(2*x**2-x+1)*(y**2+3*y-1)*(-z+2),(x**2-x+1)*(y+1)*(2*z**2-z+2)],
                  [(2*x**2-x+1)*(y**2+3*y-1)*(-z+2),(x+1)*(4*y**3-y**2+2*y-1)*(z+1),(-x+4)*(-y**2+3*y-1)*(-2*z**2-z+4)],
                  [(x**2-x+1)*(y+1)*(2*z**2-z+2),(-x+4)*(-y**2+3*y-1)*(-2*z**2-z+4),(x+4)*(y-1)*(-z**3-2*z**2-z+4)]])
P3 = sp.Matrix([[57*x*y*z + 151*x*y + 2*x*z + 16*x + 72*y*z + 42*y - 80*z - 28]])
P0Alt = sp.Matrix([[(x-2)*(y-3)*(z-4)],
                   [(3*x+1)*(2*y-1)*(z+1)],
                   [(x+1)*(y-1)*(z+3)]])
P1Alt = sp.Matrix([[(x-2)*(y-3)*(z-4),(x-2)*(y-3)*(z-4),(x-2)*(y-3)*(z-4)],
                   [(3*x+1)*(2*y-1)*(z+1),(x+1)*(y-1)*(z+3),(x+1)*(y-1)*(z+3)],
                   [(x+1)*(y-1)*(z+3),(3*x+1)*(2*y-1)*(z+1),-(x-2)*(y-3)*(z-4)-(x+1)*(y-1)*(z+3)]])
P2Alt = sp.Matrix([[(x-2)*(y-3)*(z-4),(3*x+1)*(2*y-1)*(z+1),(x-2)*(y-3)*(z-4)],
                   [(3*x+1)*(2*y-1)*(z+1),(x+1)*(y-1)*(z+3),(3*x+1)*(2*y-1)*(z+1)],
                   [(x-2)*(y-3)*(z-4),(3*x+1)*(2*y-1)*(z+1),(x+4)*(y-1)*(z+1)]])
P3Alt = sp.Matrix([[5*x*y*z + 146*x*y + 3*x*z + 6*x + 7*y*z + 2*y - 3*z - 1]])


In [ ]:
print0(P0)

In [ ]:
print1(P1)

In [ ]:
print2(P2)

In [ ]:
print0(P0Alt)

In [ ]:
print1(P1Alt)

In [ ]:
print2(P2Alt)

In [ ]:
print(sp.ccode(L2(P0,P0)))
print(sp.ccode(L2(P0,P0Alt)))
print(sp.ccode(L2(P1,P1)))
print(sp.ccode(L2(P1,P1Alt)))
print(sp.ccode(L2(P2,P2)))
print(sp.ccode(L2(P2,P2Alt)))
print(sp.ccode(L2(P3,P3)))
print(sp.ccode(L2(P3,P3Alt)))

In [ ]:
def print1adj(f):
    df = -div(f)
    print("  static double f(unsigned i, unsigned j, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(f)
    print("  }\n  static double deltaf(unsigned i, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(df)
    print("  }")
def print2adj(f):
    df = curl(f)
    print("  static double f(unsigned i, unsigned j, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(f)
    print("  }\n  static double deltaf(unsigned i, unsigned j, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(df)
    print("  }")
def print3adj(f):
    df = hessian(f)
    print("  static double f(const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    print(ADB,cppcode(f),ADE,sep='')
    print("  }\n  static double deltaf(unsigned i, unsigned j, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(df)
    print("  }")

In [ ]:
P1C0 = sp.Matrix([[0,1,2],[0,0,0],[0,0,0]])
P1C1 = sp.Matrix([[(2*x**2-x+1)*(y**2+3*y-1)*(2*z**2-z+2),(x**3-2*x**2-x+1)*(3*y-1)*(2*z**2-z+2),(2*x**3+x**2-x+1)*(y**2+3*y-1)*(z+2)],
                  [(x+1)*(4*y**3-y**2+2*y-1)*(3*z**2-z+3),(3*x**2-x+1)*(-y**2+1*y-1)*(3*z**2-z+3),(5*x**2-x+1)*(4*y**3-y**2+2*y-1)*(z-1)],
                  [(-x+4)*(-y**2+3*y-1)*(-z**3-2*z**2-z+3),(2*x**2-x+4)*(y-1)*(-z**3-2*z**2-z+4),-(2*x**2-x+1)*(y**2+3*y-1)*(2*z**2-z+2)-(3*x**2-x+1)*(-y**2+1*y-1)*(3*z**2-z+3)]])
P1C2 = sp.Matrix([[sp.sin(y)+sp.cos(x),sp.exp(z),2*x+1],[sp.sin(y),-sp.cos(x),0],[0,0,-sp.sin(y)]])
P2C0 = sp.Matrix([[0,1,2],[1,0,0],[2,0,3]])
P2C1 = sp.Matrix([[(3*x**3+2*x**2-x+1)*(3*y-1)*(z+2),(2*x**2-x+1)*(y**2+3*y-1)*(-z+2),(x**2-x+1)*(y+1)*(2*z**2-z+2)],
                  [(2*x**2-x+1)*(y**2+3*y-1)*(-z+2),(x+1)*(4*y**3-y**2+2*y-1)*(z+1),(-x+4)*(-y**2+3*y-1)*(-2*z**2-z+4)],
                  [(x**2-x+1)*(y+1)*(2*z**2-z+2),(-x+4)*(-y**2+3*y-1)*(-2*z**2-z+4),(x+4)*(y-1)*(-z**3-2*z**2-z+4)]])
P2C2 = sp.Matrix([[sp.sin(y)+sp.cos(x),sp.exp(z),2*x+1],[sp.exp(z),0,0],[2*x+1,0,sp.sin(y)]])
P3C0 = 3*x/x
P3C1 = 57*x*y*z + 151*x*y + 2*x*z + 16*x + 72*y*z + 42*y - 80*z - 28
P3C2 = sp.sin(y)*sp.cos(z)*x

BB = x*(1-x)*y*(1-y)*z*(1-z)
P1H0 = sp.Matrix([[BB,y*(1-y),z*(1-z)],
                  [x*(1-x),BB,z*(1-z)],
                  [x*(1-x),y*(1-y),BB]])
P2H0 = sp.Matrix([[y*(1-y)*z*(1-z),BB,BB],
                  [BB,x*(1-x)*z*(1-z),BB],
                  [BB,BB,x*(1-x)*y*(1-y)]])


In [ ]:
print("struct P1C0 {")
print1adj(P1C0)
print("};\nstruct P1C1 {")
print1adj(P1C1)
print("};\nstruct P1C2 {")
print1adj(P1C2)
print("};\nstruct P2C0 {")
print2adj(P2C0)
print("};\nstruct P2C1 {")
print2adj(P2C1)
print("};\nstruct P2C2 {")
print2adj(P2C2)
print("};\nstruct P3C0 {")
print3adj(P3C0)
print("};\nstruct P3C1 {")
print3adj(P3C1)
print("};\nstruct P3C2 {")
print3adj(P3C2)
print("};")

In [ ]:
print("struct P1A0 {")
print1adj(sp.hadamard_product(P1C2,P1H0))
print("};\nstruct P2A0 {")
print2adj(sp.hadamard_product(P2C2,P2H0))
print("};\nstruct P3A0 {")
print3adj(P3C2*BB)
print("};")

In [ ]:
print("struct P1A1 {")
print1adj(P1C0*BB*BB)
print("};\nstruct P2A1 {")
print2adj(P2C0*BB*BB)
print("};\nstruct P3A1 {")
print3adj(P3C0*BB*BB)
print("};")